In [3]:
# Import libraries
import pandas as pd
import numpy as np

In [11]:
# Load data
df = pd.read_excel('/Users/dhari/Downloads/Group Project Data-SFD Data-Toronto Police.xlsx') 

# Print the first few rows of the dataframe to ensure it's loaded correctly
print(df.head())

           X          Y  OBJECTID EVENT_UNIQUE_ID                OCC_DATE   
0 -79.234955  43.781528         1   GO-2004397105  2004/05/15 04:00:00+00  \
1 -79.618218  43.733547         2   GO-2004600109  2004/07/26 04:00:00+00   
2 -79.518759  43.769157         3   GO-2004311851  2004/05/14 04:00:00+00   
3 -79.378870  43.660665         4   GO-2004736004  2004/12/19 05:00:00+00   
4 -79.212435  43.812075         5   GO-2004303455  2004/04/12 04:00:00+00   

   OCC_YEAR OCC_MONTH   OCC_DOW  OCC_DOY  OCC_DAY  ...  OCC_TIME_RANGE   
0      2004       May  Saturday      136       15  ...           Night  \
1      2004      July    Monday      208       26  ...         Evening   
2      2004       May    Friday      135       14  ...       Afternoon   
3      2004  December    Sunday      354       19  ...           Night   
4      2004     April    Monday      103       12  ...         Evening   

  DIVISION DEATH  INJURIES  HOOD_158       NEIGHBOURHOOD_158 HOOD_140   
0      D41     0   

In [5]:
# Let's see the data types of all the columns
print(df.dtypes)

X                    float64
Y                    float64
OBJECTID               int64
EVENT_UNIQUE_ID       object
OCC_DATE              object
OCC_YEAR               int64
OCC_MONTH             object
OCC_DOW               object
OCC_DOY                int64
OCC_DAY                int64
OCC_HOUR               int64
OCC_TIME_RANGE        object
DIVISION              object
DEATH                  int64
INJURIES               int64
HOOD_158              object
NEIGHBOURHOOD_158     object
HOOD_140              object
NEIGHBOURHOOD_140     object
LONG_WGS84           float64
LAT_WGS84            float64
dtype: object


Step 1: Drop Unnecessary Columns 

1) 'X, Y, LONG_WGS84, LAT_WGS84' - These are all geographical coordinates. X and Y likely represent a projected coordinate system, while LONG_WGS84 and LAT_WGS84 represent a geographic (longitude and latitude) coordinate system. For our analysis on spatial patterns and neighborhood grouping, we may need the longitude and latitude data. The X and Y could possibly be redundant if they represent the same locations.

2) 'OBJECTID' - This is likely a unique identifier for each row or event. 
It might not provide much meaningful information for our analysis.

3) 'EVENT_UNIQUE_ID' - This might be a unique identifier for each event. 
If similar to OBJECTID, it might not be directly useful for our analysis.

4) 'OCC_DATE, OCC_YEAR, OCC_MONTH, OCC_DOW, OCC_DOY, OCC_DAY, OCC_HOUR, OCC_TIME_RANGE' - These are all related to the time of the incidents. 
They will be crucial to our time pattern analysis.

5) 'DIVISION' - This helps understand which police division is incharge of the criminal-offence.
This could be useful for understanding patterns across different police divisions.

6) 'DEATH, INJURIES' - These could be important for our severity analysis.

7) 'HOOD_158, NEIGHBOURHOOD_158, HOOD_140, NEIGHBOURHOOD_140' - These columns seem to provide neighborhood information under two different structures. 
We could potentially choose one structure to use depending on which is more relevant or comprehensive for our analysis. 

8) 'OCC_DATE' - This column provides the date when the incident occurred. 
As we have separate columns detailing the year (OCC_YEAR), month (OCC_MONTH), day of the week (OCC_DOW), day of the year (OCC_DOY), day of the month (OCC_DAY), and hour of the day (OCC_HOUR) of each incident, the OCC_DATE column becomes redundant and can be dropped.
Having the date broken down into these separate features can be more useful for our analysis as it allows us to easily examine trends and patterns over time at different granularities (e.g., yearly trends, monthly trends, daily patterns, etc.). It also saves us the trouble of having to parse the date column into these separate features.

Based on this, it seems like we could potentially drop the X, Y, OBJECTID, EVENT_UNIQUE_ID columns if they don't provide additional meaningful information. We could also choose one set of neighborhood structure columns to keep (HOOD_158, NEIGHBOURHOOD_158 or HOOD_140, NEIGHBOURHOOD_140, OCC_DATE).


In [12]:
# Drop columns
df = df.drop(['X', 'Y', 'OBJECTID', 'EVENT_UNIQUE_ID', 'HOOD_140', 'NEIGHBOURHOOD_140', 'OCC_DATE'], axis=1)

Step 2: Check Data Types

In [13]:
print(df.dtypes)

OCC_YEAR               int64
OCC_MONTH             object
OCC_DOW               object
OCC_DOY                int64
OCC_DAY                int64
OCC_HOUR               int64
OCC_TIME_RANGE        object
DIVISION              object
DEATH                  int64
INJURIES               int64
HOOD_158              object
NEIGHBOURHOOD_158     object
LONG_WGS84           float64
LAT_WGS84            float64
dtype: object


'DIVISION': This column is of object data type and if this column is meant to be categorical (i.e., it represents distinct categories or groups), it's good practice to convert this column to a category data type. 

This can save memory and also allows us to use categorical functions.

In [14]:
# Convert object data type to categorical 
df['DIVISION'] = df['DIVISION'].astype('category')

'OCC_MONTH and OCC_DOW': These columns represent the month and day of the week of the offence respectively. Currently, they are stored as object types. We should consider converting them into a category type. 

This would allow for logical ordering of these fields, which could be helpful for analysis.

**
OCC_MONTH: This column represents the month of the year (12 unique values). Converting this to a categorical type allows for logical ordering (January, February, March, etc.) which is beneficial for some types of analysis, especially those involving time series or trends.

OCC_DOW: This column represents the day of the week (7 unique values). Similar to the month, changing this to a categorical type allows for logical ordering (Monday, Tuesday, Wednesday, etc.) which can be helpful for analyzing weekly trends.

OCC_YEAR: This column is already an integer type which is efficient for storing and processing. While it could be converted to a categorical type, it's generally more efficient to keep it as an integer for any kind of mathematical operations (like calculating the number of years between incidents, for example).

OCC_DAY: This column represents the day of the month (1 to 31). Like OCC_YEAR, it's also an integer type which is appropriate for this kind of data. While there is a logical ordering to the days of the month, converting this column to a categorical type wouldn't provide a significant benefit because numerical operations with day values are quite common (like calculating the number of days between incidents, for example).
**

In [16]:
df['OCC_MONTH'] = df['OCC_MONTH'].astype('category')
df['OCC_DOW'] = df['OCC_DOW'].astype('category')

'HOOD_158 and NEIGHBOURHOOD_158': These columns represent identifiers and names of neighbourhoods using City of Toronto's new 158 neighbourhood structure. It seems they could also be converted to category type since they contain a limited number of unique values. 
While they are currently treated as objects, changing them to a categorical type can be beneficial for several reasons.

In [17]:
df['HOOD_158'] = df['HOOD_158'].astype('category')
df['NEIGHBOURHOOD_158'] = df['NEIGHBOURHOOD_158'].astype('category')

'OCC_TIME_RANGE': seems to contain specific time ranges during which an offense occurred. Since the number of unique time ranges is limited, it would be more efficient to convert this column from an 'object' type to a 'category' type.
Categories are much more memory efficient and operations on categories are usually faster than 'object'.

In [19]:
df['OCC_TIME_RANGE'] = df['OCC_TIME_RANGE'].astype('category')

In [20]:
# Check data types to make sure they are correct
print(df.dtypes)

OCC_YEAR                int64
OCC_MONTH            category
OCC_DOW              category
OCC_DOY                 int64
OCC_DAY                 int64
OCC_HOUR                int64
OCC_TIME_RANGE       category
DIVISION             category
DEATH                   int64
INJURIES                int64
HOOD_158             category
NEIGHBOURHOOD_158    category
LONG_WGS84            float64
LAT_WGS84             float64
dtype: object


Data types seem to all be correct based on the analysis purpose:
1) OCC_YEAR, OCC_DOY, OCC_DAY, OCC_HOUR, DEATH, and INJURIES are of integer type (int64), which is suitable for numerical and count data.

2) OCC_MONTH, OCC_DOW, OCC_TIME_RANGE, DIVISION, HOOD_158, and NEIGHBOURHOOD_158 are of category type, which is suitable for limited and usually fixed numbers of possible values. Changing these from 'object' to 'category' should save memory and potentially increase computational efficiency.

3) LONG_WGS84 and LAT_WGS84 are of float type (float64), which is suitable for numerical data that can take any real-numbered value.

Step 3: Check Missing Values

In [21]:
df.isnull().sum()

OCC_YEAR             0
OCC_MONTH            0
OCC_DOW              0
OCC_DOY              0
OCC_DAY              0
OCC_HOUR             0
OCC_TIME_RANGE       0
DIVISION             0
DEATH                0
INJURIES             0
HOOD_158             0
NEIGHBOURHOOD_158    0
LONG_WGS84           0
LAT_WGS84            0
dtype: int64

That's great news! 
This output indicates that there are no missing values in our dataset. Every cell in each column has some data. This is an ideal situation as it often simplifies further data preprocessing and analysis.

Step 4: Outlier Detection

This step involves identifying outliers in your data. An outlier is an observation that deviates significantly from other observations. They can occur due to variability in the data or due to measurement errors.

However, in the context of our dataset, which involves data on shootings and firearm discharges, outliers could signify important real-world events, such as mass shootings or other significant incidents. Therefore, the term "outlier" does not necessarily indicate a problem in this context, but rather data points of particular interest or concern and instead of eliminating these outliers, our analysis would focus on understanding their cause and impact.

Step 5: Feature Engineering

Seasonality in data can often reveal important insights. For example, in the context of this dataset, it may be interesting to investigate if more shooting incidents occur in certain seasons (Spring, Summer, Autumn, Winter) than others. This could be due to various reasons such as changes in weather, school vacations, holiday periods etc.

For instance, if there are more incidents in the summer, it could be due to the fact that people tend to be outside more often in warmer weather, leading to more potential conflicts. On the other hand, fewer incidents in the winter could be attributed to fewer people being outside due to cold weather.

In order to carry out this analysis, we can create a new column 'OCC_SEASON' based on the 'OCC_MONTH' column.

In [23]:
# Add a season column to dataset
def month_to_season(month):
    if month in ['March', 'April', 'May']:
        return 'Spring'
    elif month in ['June', 'July', 'August']:
        return 'Summer'
    elif month in ['September', 'October', 'November']:
        return 'Autumn'
    else:
        return 'Winter'

df['OCC_SEASON'] = df['OCC_MONTH'].apply(month_to_season)


In [24]:
# View dataset
df

,OCC_YEAR,OCC_MONTH,OCC_DOW,OCC_DOY,OCC_DAY,OCC_HOUR,OCC_TIME_RANGE,DIVISION,DEATH,INJURIES,HOOD_158,NEIGHBOURHOOD_158,LONG_WGS84,LAT_WGS84,OCC_SEASON
0,2004,May,Saturday,136,15,1,Night,D41,0,0,142,Woburn North,-79.234955,43.781528,Spring
1,2004,July,Monday,208,26,23,Evening,D23,0,0,1,West Humber-Clairville,-79.618218,43.733547,Summer
2,2004,May,Friday,135,14,16,Afternoon,D31,1,1,24,Black Creek,-79.518759,43.769157,Spring
3,2004,December,Sunday,354,19,0,Night,D51,1,0,168,Downtown Yonge East,-79.378870,43.660665,Winter
4,2004,April,Monday,103,12,22,Evening,D42,0,0,146,Malvern East,-79.212435,43.812075,Spring
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5702,2022,June,Thursday,153,2,3,Night,D33,0,1,53,Henry Farm,-79.362941,43.771764,Summer
5703,2022,July,Sunday,198,17,3,Night,D14,1,1,164,Wellington Place,-79.402702,43.643920,Summer
5704,2022,February,Monday,52,21,5,Night,D43,0,0,135,Morningside,-79.196217,43.786621,Winter
5705,2022,September,Sunday,254,11,1,Night,D11,0,2,83,Dufferin Grove,-79.443993,43.658083,Autumn


Conclusion: 

The dataset has undergone a comprehensive cleaning and data processing process, resulting in a well-prepared dataset that is ready for analysis. Here is a summary of the key steps and improvements made:

Step 1: Drop Unnecessary Columns: Several columns, including 'X', 'Y', 'OBJECTID', 'EVENT_UNIQUE_ID', 'HOOD_140', 'NEIGHBOURHOOD_140', and 'OCC_DATE', were identified as unnecessary for our analysis and were dropped. This improved the focus and clarity of the dataset.

Step 2: Check Data Types: The data types of certain columns were evaluated and adjusted as needed. 'DIVISION', 'OCC_MONTH', 'OCC_DOW', 'HOOD_158', 'NEIGHBOURHOOD_158', and 'OCC_TIME_RANGE' were converted to the 'category' data type, which not only saves memory but also allows for efficient categorical operations.

Step 3: Check Missing Values: No missing values were found in the dataset, ensuring data completeness and minimizing the need for imputation or data filling techniques.

Step 4: Outlier Detection: Given the nature of the dataset (shootings and firearm incidents), outliers were not treated as anomalies but rather as significant events that hold crucial information for analysis. Therefore, no further outlier removal or manipulation was performed.

Step 5: Feature Engineering: Additional features were created to enhance the dataset's analytical value. The 'OCC_SEASON' column was derived from the 'OCC_MONTH' column, categorizing the months into seasons (Spring, Summer, Autumn, Winter). This enables exploration of potential seasonal patterns in shooting incidents.

The dataset is now cleaned, structured, and prepared to facilitate insightful analysis. By eliminating unnecessary columns, optimizing data types, handling missing values, and incorporating additional features, the dataset has become more focused, efficient, and informative for our analysis purpose.

These cleaning and processing steps ensure that the dataset is in a suitable format to apply various analytical techniques, such as exploring temporal patterns, investigating neighborhood-specific trends, and conducting statistical modeling. The efforts invested in data preparation are crucial for producing reliable and meaningful insights from the analysis.